<a href="https://colab.research.google.com/github/giacomosansoni/Siamese_Network_Artist_Similarity/blob/main/min_max_scaling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import os

In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/My Drive/Thesis/Thesis/

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive/.shortcut-targets-by-id/1D4FFhBTYn6QfBuxB5OXjn7JgI3z14YxH/Thesis


In [ ]:
root_dir = os.getcwd()
emb_dir = os.path.join(root_dir, 'Embeddings_40')
with np.load(os.path.join(emb_dir, 'ffe54de6-5cee-4c0f-8329-f5946f9bc3ff.npz')) as data:
    ids = data['ids']
    img_embb = data['img_embeddings']
    emb1_1 = data['embeddings1_1']
    emb1_2 = data['embeddings1_2']
    emb2_1 = data['embeddings2_1']
    emb2_2 = data['embeddings2_2']

ids.shape, img_embb.shape, emb1_1.shape, ids.dtype, img_embb.dtype, emb1_1.dtype

((40,),
 (40, 768),
 (40, 768),
 dtype('<U18'),
 dtype('float16'),
 dtype('float32'))

In [ ]:
min_img = np.full((768,), 50, dtype=np.float16)
max_img = np.full((768,), -50, dtype=np.float32)
min_emb1_1 = np.full((768,), 50, dtype=np.float32)
max_emb1_1 = np.full((768,), -50, dtype=np.float32)
min_emb1_2 = np.full((768,), 50, dtype=np.float32)
max_emb1_2 = np.full((768,), -50, dtype=np.float32)
min_emb2_1 = np.full((768,), 50, dtype=np.float32)
max_emb2_1 = np.full((768,), -50, dtype=np.float32)
min_emb2_2 = np.full((768,), 50, dtype=np.float32)
max_emb2_2 = np.full((768,), -50, dtype=np.float32)

# First pass to determine min and max values using vectorized operations
for file_name in tqdm(os.listdir(emb_dir)):
    if file_name.endswith('.npz'):
        file_path = os.path.join(emb_dir, file_name)
        with np.load(file_path) as data:
            img_emb = data['img_embeddings']
            emb1_1 = data['embeddings1_1']
            emb1_2 = data['embeddings1_2']
            emb2_1 = data['embeddings2_1']
            emb2_2 = data['embeddings2_2']

            # Update min and max values using vectorized operations
            min_img = np.minimum(min_img, np.min(img_emb, axis=0))
            max_img = np.maximum(max_img, np.max(img_emb, axis=0))

            min_emb1_1 = np.minimum(min_emb1_1, np.min(emb1_1, axis=0))
            max_emb1_1 = np.maximum(max_emb1_1, np.max(emb1_1, axis=0))

            min_emb1_2 = np.minimum(min_emb1_2, np.min(emb1_2, axis=0))
            max_emb1_2 = np.maximum(max_emb1_2, np.max(emb1_2, axis=0))

            min_emb2_1 = np.minimum(min_emb2_1, np.min(emb2_1, axis=0))
            max_emb2_1 = np.maximum(max_emb2_1, np.max(emb2_1, axis=0))

            min_emb2_2 = np.minimum(min_emb2_2, np.min(emb2_2, axis=0))
            max_emb2_2 = np.maximum(max_emb2_2, np.max(emb2_2, axis=0))

100%|██████████| 3992/3992 [04:25<00:00, 15.05it/s] 


In [ ]:
emb_dir = os.path.join(root_dir, 'Embeddings_40')

# Second pass to apply min-max scaling and save the scaled embeddings
def min_max_scale(x, min_val, max_val):
    return (x - min_val) / (max_val - min_val)

count = 0
for file_name in tqdm(os.listdir(emb_dir)):
    if file_name.endswith('.npz'):
        file_path = os.path.join(emb_dir, file_name)
        with np.load(file_path) as data:
            ids = data['ids']
            img_emb = data['img_embeddings']
            emb1_1 = data['embeddings1_1']
            emb1_2 = data['embeddings1_2']
            emb2_1 = data['embeddings2_1']
            emb2_2 = data['embeddings2_2']

        # Apply min-max scaling using vectorized operations
        scaled_img_emb = min_max_scale(img_emb, min_img, max_img).astype(np.float16)
        scaled_emb1_1 = min_max_scale(emb1_1, min_emb1_1, max_emb1_1).astype(np.float16)
        scaled_emb1_2 = min_max_scale(emb1_2, min_emb1_2, max_emb1_2).astype(np.float16)
        scaled_emb2_1 = min_max_scale(emb2_1, min_emb2_1, max_emb2_1).astype(np.float16)
        scaled_emb2_2 = min_max_scale(emb2_2, min_emb2_2, max_emb2_2).astype(np.float16)

        # Save the scaled embeddings back to a new .npz file or overwrite the original
        save_path = os.path.join(root_dir, 'Embeddings_40_scaled', file_name)
        np.savez(save_path,
                  ids = ids,
                  img_embeddings=scaled_img_emb,
                  embeddings1_1=scaled_emb1_1,
                  embeddings1_2=scaled_emb1_2,
                  embeddings2_1=scaled_emb2_1,
                  embeddings2_2=scaled_emb2_2)
        count+=1

count

100%|██████████| 3992/3992 [02:59<00:00, 22.28it/s]


3992

In [ ]:
# CHECK OF SOME MISSING .npz FILES
import os
from tqdm import tqdm

# Define the paths to the directories
directory1 = os.path.join(root_dir, 'instagram')
directory2 = os.path.join(root_dir, 'Embeddings_40_scaled')

# Get a list of folders in directory1
folders = [folder for folder in os.listdir(directory1)
           if os.path.isdir(os.path.join(directory1, folder))]

# Get a list of .npz files in directory2
npz_files = [file for file in os.listdir(directory2)
             if file.endswith('.npz')]

# Create a set of the base names of the npz files without the extension
npz_file_names = set(os.path.splitext(file)[0] for file in npz_files)

# Check each folder against the npz files
for folder in tqdm(folders):
    if not (folder in npz_file_names):
        print(f"No match found: Folder '{folder}' does not have a corresponding .npz file in directory2.")

100%|██████████| 3995/3995 [00:00<00:00, 572100.26it/s]

No match found: Folder '3f23c754-8049-44bd-92ca-44a4bacb4380' does not have a corresponding .npz file in directory2.
No match found: Folder '08870a97-6af5-4db8-97fa-859bb9f2e0c2' does not have a corresponding .npz file in directory2.
No match found: Folder 'cdcd53c9-f37c-48cc-b7e3-dfe34be22e92' does not have a corresponding .npz file in directory2.
No match found: Folder '1550f952-c91b-40d7-9b4d-d26a259ee932' does not have a corresponding .npz file in directory2.


In [ ]:
#CHECK OF THE SHAPES OF EVERY .npz FILE

import os
import numpy as np

# Specify the directory containing the .npz files
emb_dir = os.path.join(root_dir, 'Embeddings_40_scaled')

# Function to check the array shapes in the npz file
def check_shapes(file_path):
    with np.load(file_path) as data:
        ids = data['ids']
        img_embb = data['img_embeddings']
        emb1_1 = data['embeddings1_1']
        emb1_2 = data['embeddings1_2']
        emb2_1 = data['embeddings2_1']
        emb2_2 = data['embeddings2_2']

        # Check the lengths of all arrays and the specific shape for emb2_2
        if (len(ids) == 40 and len(img_embb) == 40 and
            len(emb1_1) == 40 and len(emb1_2) == 40 and
            len(emb2_1) == 40 and len(emb2_2) == 40 and
            emb2_2.shape == (40, 768)):
            return True
        else:
            return False

# Check each .npz file in the directory
count = 0
for filename in os.listdir(emb_dir):
    if filename.endswith('.npz'):
        count+=1
        file_path = os.path.join(emb_dir, filename)
        if not check_shapes(file_path):
            print(f"{filename}: Shapes are NOT correct.")

count

3992